In [ ]:
from langchain import PromptTemplate
from langchain.chain import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveChararcterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [ ]:
PINECONE_API_KEY=”1a235cd3-71a9-46af-9cd6-9867ac1028a1”
PINECONE_API_ENV=” ”


In [ ]:
#EXTRACT DATA FROM PDF
def load_pdf(data):
         loader=DirectoryLoader(data,glob=”*.pdf”,Loader_cls=PyPDFLoader)
         documents=loader.load()
         return documents


In [ ]:
extracted_data= load_pdf("/data")

In [ ]:
#To create text chunks
def text_split(extracted_dadta):
    text_splitter=RecursiveChararcterTextSplitter(chunk_size = 500,chunk_overlap = 20)
    text_splitter.split_documents(extracted_data)

    return text_chunks

In [ ]:
#chunks
text_chunks = text_split(extracted)
print("length of my chunk:",len(text_chunks))

In [ ]:
#To download the model
embeddings = download_hugging_face_embeddings()

In [ ]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name= "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
#Test the embedding(if its able to convert text to vectors)
query_result = embeddings.embed_query("Hello World")
print("Length",len(query_result))



In [ ]:
#applying embedding on data
pinecone.init(api_key=PINECONE_API_KEY,envirnment=PINE_API_ENV)

index_name="medical-chatbot"

In [ ]:
#Create Embeddings for Each of the Text Chunks and Sorting
docsearch = Pinecone.from_text([t.page_conent for t in text_chunks],embeddings,index_name=index_name)

In [ ]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query  = "What are allergies"

docs=docsearch.similarity_search(query, k=3)

print("Print",docs)

In [ ]:
Prompt_template="""
use the following pieces of information to answer the user's question.
If you don't know the answer,just say that you don't know,dont try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer :
"""

In [ ]:
PROMPT=PromptTemplate(template=Prompt_template, input_variables=["context","question"])
chain_type_kwargs={"prompt":PROMPT}

In [ ]:
llm = CTransformers(model="model/Llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llma",
                    config={'max_new_tokens':512,
                            'temperature':0.8}

)

In [ ]:
#Create question-answer object
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuffs",
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_kwargs=chain_type_kwargs
)


In [ ]:
#Test to give the optimum answer
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response :",result[["result"]])